<a href="https://colab.research.google.com/github/charlie9578/wind-turbine-kml/blob/master/windTurbineModelKML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install simplekml
!pip install pycollada
!pip install -U ipykernel

  Using cached ipykernel-6.29.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached pyzmq-25.1.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (4.9 kB)
Using cached ipykernel-6.29.0-py3-none-any.whl (116 kB)
Using cached pyzmq-25.1.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: pyzmq
    Found existing installation: pyzmq 23.2.0
    Uninstalling pyzmq-23.2.0:
      Successfully uninstalled pyzmq-23.2.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 6.25.0
    Uninstalling ipykernel-6.25.0:
      Successfully uninstalled ipykernel-6.25.0


In [2]:
import simplekml
kml = simplekml.Kml()
kml.newpoint(name="Kirstenbosch", coords=[(18.432314,-33.988862)])
kml.save("botanicalgarden.kml")

In [3]:
import collada

In [4]:
mesh = collada.Collada()
effect = collada.material.Effect("effect0", [], "phong", diffuse=(1,0,0), specular=(0,1,0))
mat = collada.material.Material("material0", "mymaterial", effect)
mesh.effects.append(effect)
mesh.materials.append(mat)

In [5]:
import numpy
vert_floats = [-50,50,50,50,50,50,-50,-50,50,50,
     -50,50,-50,50,-50,50,50,-50,-50,-50,-50,50,-50,-50]
normal_floats = [0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,
     0,1,0,0,1,0,0,1,0,0,-1,0,0,-1,0,0,-1,0,0,-1,0,-1,0,0,
     -1,0,0,-1,0,0,-1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,-1,
     0,0,-1,0,0,-1,0,0,-1]
vert_src = collada.source.FloatSource("cubeverts-array", numpy.array(vert_floats), ('X', 'Y', 'Z'))
normal_src = collada.source.FloatSource("cubenormals-array", numpy.array(normal_floats), ('X', 'Y', 'Z'))

In [6]:
geom = collada.geometry.Geometry(mesh, "geometry0", "mycube", [vert_src, normal_src])

To add a triangle set to the geometry, we can call the Geometry.createTriangleSet() method. To do this, we need to define the inputs to the triangle set. In this case, we are going to input the arrays we previously defined:

In [7]:
input_list = collada.source.InputList()
input_list.addInput(0, 'VERTEX', "#cubeverts-array")
input_list.addInput(1, 'NORMAL', "#cubenormals-array")

This says to use the source with identifier cubeverts-array as the vertex source and source with identifier cubenormals-array as the normal source. The offsets indicate that the vertex data is the first offset in the index array and the normal data is the second offset in the index array. Let’s now create the index array:

In [8]:
indices = numpy.array([0,0,2,1,3,2,0,0,3,2,1,3,0,4,1,5,5,6,0,
     4,5,6,4,7,6,8,7,9,3,10,6,8,3,10,2,11,0,12,
     4,13,6,14,0,12,6,14,2,15,3,16,7,17,5,18,3,
     16,5,18,1,19,5,20,7,21,6,22,5,20,6,22,4,23])

Now that we have an index array, an input list, and a material, we can create a triangle set and add it to the geometry’s list of primitives. We then add it to the list of geometries in the mesh:

In [9]:
triset = geom.createTriangleSet(indices, input_list, "materialref")
geom.primitives.append(triset)
mesh.geometries.append(geom)

We now have everything we need in the object except for a scene. To get the geometry to show up, we have to create a scene. First, we instantiate the geometry into a scene node, mapping it to a material:


In [10]:
matnode = collada.scene.MaterialNode("materialref", mat, inputs=[])
geomnode = collada.scene.GeometryNode(geom, [matnode])
node = collada.scene.Node("node0", children=[geomnode])

Now that we have the scene node, we can create a scene, add the node to the scene, add the scene to the document, and then set our scene as the default:


In [11]:
myscene = collada.scene.Scene("myscene", [node])
mesh.scenes.append(myscene)
mesh.scene = myscene

We can now save the document to a file:

In [12]:
mesh.write('test.dae')

If you load this file, it should look like a red cube. Here’s a screenshot:

In [13]:
import simplekml
kml = simplekml.Kml()
#kml.newpoint(name="Kirstenbosch", coords=[(18.432314,-33.988862)])
#netlink = kml.newnetworklink(name="Network Link")
#netlink.link.href = 'test.dae' #"http://simplekml.googlecode.com/hg/samples/samples.kml"
model = kml.newmodel(name='cube',)
model.link.href = 'https://github.com/charlie9578/wind-turbine-kml/blob/master/test.dae'
model.altitudemode = 'clampToGround'
model.location.latitude = 52
model.location.longitude = 0
model.scale.x = 1000
model.scale.y = 1000
model.scale.z = 1000
kml.save("botanicalgarden4.kml")